In [68]:
import numpy as np
import pandas as pd
import tracktor as tr
import cv2
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist

### Global parameters
This cell (below) enlists user-defined parameters

In [73]:
# colours is a vector of BGR values which are used to identify individuals in the video
# since we only have one individual, the program will only use the first element from this array i.e. (0,0,255) - red
# number of elements in colours should be greater than n_inds (THIS IS NECESSARY FOR VISUALISATION ONLY)
n_inds = 1
colours = [(0,0,255),(0,255,255),(255,0,255),(255,255,255),(255,255,0),(255,0,0),(0,255,0),(0,0,0)]

# this is the block_size and offset used for adaptive thresholding (block_size should always be odd)
# these values are critical for tracking performance
block_size = 81
offset = 40

# minimum area and maximum area occupied by the animal in number of pixels
# this parameter is used to get rid of other objects in view that might be hard to threshold out but are differently sized
min_area = 1000
max_area = 10000

# mot determines whether the tracker is being used in noisy conditions to track a single object or for multi-object
# using this will enable k-means clustering to force n_inds number of animals
mot = False

# name of source video and paths
video = 'fish_video'
input_vidpath = '/home/user/Documents/Vivek/tracktor/videos/' + video + '.mp4'
output_vidpath = '/home/user/Documents/Vivek/tracktor/output/' + video + '.mp4'
output_filepath = '/home/user/Documents/Vivek/tracktor/output/' + video + '.csv'
output_framesize = (1920, 1080)
codec = 'DIVX' # try other codecs if the default doesn't work ('DIVX', 'avc1', 'XVID') note: this list is non-exhaustive

The cell below runs the tracking code

In [74]:
## Path to source video
cap = cv2.VideoCapture(input_vidpath)

## Video writer class to output video with contour and centroid of tracked object(s)
# make sure the frame size matches size of array 'final'
fourcc = cv2.VideoWriter_fourcc(*codec)
out = cv2.VideoWriter(filename = output_vidpath, fourcc = fourcc, fps = 30.0, frameSize = output_framesize, isColor = True)

## Individual location(s) measured in the last and current step
meas_last = list(np.zeros((n_inds,2)))
meas_now = list(np.zeros((n_inds,2)))

last = 0
df = []

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    this = cap.get(1)
    if ret == True:
        frame = cv2.resize(frame, None, fx = 1.0, fy = 1.0, interpolation = cv2.INTER_LINEAR)
        thresh = tr.colour_to_thresh(frame, block_size, offset)
        final, contours, meas_last, meas_now = tr.detect_and_draw_contours(frame, thresh, meas_last, meas_now, min_area, max_area)
        row_ind, col_ind = tr.hungarian_algorithm(meas_last, meas_now)
        final, meas_now, df = tr.reorder_and_draw(final, colours, n_inds, col_ind, meas_now, df, mot, this)
        
        # Create output dataframe
        for i in range(n_inds):
            df.append([this, meas_now[i][0], meas_now[i][1]])
        
        # Display the resulting frame
        out.write(final)
        cv2.imshow('frame', final)
        if cv2.waitKey(1) == 27 or meas_now[0][0] < 100 or meas_now[0][0] > cap.get(3) - 100 or meas_now[0][1] < 100 or meas_now[0][1] > cap.get(4) - 100:
            break
            
    if last == this:
        break
    
    last = this

## Write positions to file
df = pd.DataFrame(np.matrix(df), columns = ['frame','pos_x','pos_y'])
df.to_csv(output_filepath, sep=',')

## When everything done, release the capture
cap.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

## Summary statistics
The cells below provide functions to perform basic summary statistics - in this case, distance moved between successive frames, cumulative distance within a time-window, velocity and acceleration.

In [58]:
df = pd.read_csv(output_filepath)
df.head()

,Unnamed: 0,frame,pos_x,pos_y
0,0,1.0,1527.810295,331.102994
1,1,2.0,1527.943607,330.730115
2,2,3.0,1527.864597,330.853322
3,3,4.0,1527.539979,331.271908
4,4,5.0,1527.189117,331.714604


In [59]:
dx = df['pos_x'] - df['pos_x'].shift(1)
dy = df['pos_y'] - df['pos_y'].shift(1)
d2x = dx - dx.shift(1)
d2y = dy - dy.shift(1)
df['speed'] = np.sqrt(dx**2 + dy**2)
df['accn'] = np.sqrt(d2x**2 + d2y**2)
df['cum_dist'] = df['speed'].cumsum()
df.head()

,Unnamed: 0,frame,pos_x,pos_y,speed,accn,cum_dist
0,0,1.0,1527.810295,331.102994,NaN,NaN,NaN
1,1,2.0,1527.943607,330.730115,0.395994,NaN,0.395994
2,2,3.0,1527.864597,330.853322,0.146364,0.539613,0.542358
3,3,4.0,1527.539979,331.271908,0.529708,0.384151,1.072066
4,4,5.0,1527.189117,331.714604,0.564876,0.035638,1.636942


In [60]:
def cumul_dist(start_fr, end_fr):
    if start_fr != 1:
        cumul_dist = df['cum_dist'][df['frame'] == end_fr].values[0] - df['cum_dist'][df['frame'] == start_fr].values[0]
    else:
        cumul_dist = df['cum_dist'][df['frame'] == end_fr].values[0]
    return cumul_dist

In [61]:
cumul_dist(150,200)

393.051675581072

Fill in the parameters below if you'd like movement measures to be converted from pixels and frames to real-world measures (cms and secs). Remember to scale pxpercm if you use a scaling factor while tracking (fx and fy arguments in cv2.resize).

In [62]:
framerate = 1000
pxpercm = 50

In [63]:
df['speed'] = df['speed'] * framerate / pxpercm
df['accn'] = df['accn'] * framerate * framerate / pxpercm
df['cum_dist'] = df['cum_dist'] / pxpercm
df.head()

,Unnamed: 0,frame,pos_x,pos_y,speed,accn,cum_dist
0,0,1.0,1527.810295,331.102994,NaN,NaN,NaN
1,1,2.0,1527.943607,330.730115,7.919873,NaN,0.007920
2,2,3.0,1527.864597,330.853322,2.927280,10792.250123,0.010847
3,3,4.0,1527.539979,331.271908,10.594168,7683.028737,0.021441
4,4,5.0,1527.189117,331.714604,11.297513,712.764347,0.032739


In [64]:
cumul_dist(150,200) / pxpercm

0.15722067023242878

In [67]:
np.nanmax(df['speed'])

916.4225774490125